# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [17]:
%%capture
%pip install matplotlib tensorflow_io "sagemaker<3" "numpy<2.0" python-dotenv awscli -U

In [11]:
import os
import sys

print(f"Using Python: {sys.executable}")

try:
    import sagemaker
    print(f"SageMaker imported from: {sagemaker.__file__}")
    from sagemaker.estimator import Estimator
    from framework import CustomFramework
    print("Imports successful!")
except ImportError as e:
    print(f"Import failed: {e}")
    print("Attempting to force reinstall sagemaker...")
    !{sys.executable} -m pip install "sagemaker<3" "numpy<2.0" --force-reinstall
    print("\nDONE. Please RESTART the kernel (circular arrow icon) and run this cell again.")

Using Python: /home/wcho/cd2688-object-detection-in-urban-environment-project-1/.venv/bin/python
SageMaker imported from: /home/wcho/cd2688-object-detection-in-urban-environment-project-1/.venv/lib/python3.10/site-packages/sagemaker/__init__.py
Imports successful!


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [12]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set the AWS region if it is not configured locally
if os.environ.get('AWS_DEFAULT_REGION') is None:
    os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

# Get the SageMaker Execution Role from .env (Best for local execution)
role = os.environ.get('SAGEMAKER_ROLE_ARN')

if role is None:
    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        print("Could not retrieve execution role via SDK. Please set SAGEMAKER_ROLE_ARN in .env")

print(role)

arn:aws:iam::305502288700:role/service-role/AmazonSageMaker-ExecutionRole-20200629T181730


In [13]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://object-detection-project/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the repository
* build the docker image and push it 
* print the container name

In [14]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.


In [15]:
# build and push the docker image. This code can be commented out after being run once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
!docker build --no-cache -t $image_name -f docker/Dockerfile.nvidiaV2 docker

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile.nvidiaV2              0.0s
 => => transferring dockerfile: 1.86kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.15.0-g  0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile.nvidiaV2              0.0s
 => => transferring dockerfile: 1.86kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.15.0-g  0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile.nvidiaV2              0.0s
 => => transferring dockerfile: 1.86kB                                     0.0s
 => [internal] load metadata for docker.

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [ ]:
# display the container name
if os.path.exists(os.path.join('docker', 'ecr_image_fullname.txt')):
    with open(os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
        container = f.readlines()[0][:-1]
container = image_name

print(container)

tf2-object-detection


In [ ]:
!docker run --rm --gpus all $container python -c "import tensorflow as tf; print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU'))); import object_detection; print('Object Detection API imported successfully')"

2026-01-28 15:28:23.539542: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-28 15:28:23.539627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-28 15:28:23.562831: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-28 15:28:23.610033: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-28 15:28:26.046425: I external/local_xla/xla/

## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be adjusted if you were to experiment with other architectures.

In [ ]:
%%bash
mkdir -p /tmp/checkpoint
mkdir -p source_dir/checkpoint
wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

--2026-01-27 18:28:34--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 192.178.223.207, 142.250.140.207, 142.250.117.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|192.178.223.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51839363 (49M) [application/x-tar]
Saving to: ‘/tmp/efficientdet.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  533K 95s
    50K .......... .......... .......... .......... ..........  0%  530K 95s
   100K .......... .......... .......... .......... ..........  0% 1.38M 75s
   150K .......... .......... .......... .......... ..........  0% 1.71M 64s
   200K .......... .......... .......... .......... ..........  0% 1.42M 58s
   250K .......... .......... .......... .......... ..........  0% 2.02M 52s
   300K .......... .......... .......... .......... .......... 

efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
efficientdet_d1_coco17_tpu-32/checkpoint/checkpoint
efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0.index


## Manual GPU Training Test

Before running the full SageMaker job, we download the data locally and run the training script manually using `docker run` to verify GPU usage and performance.

In [ ]:
%%bash
# Download data for local training test
mkdir -p data/train data/val
aws s3 cp s3://cd2688-object-detection-tf2/train/ data/train/ --recursive --quiet
aws s3 cp s3://cd2688-object-detection-tf2/val/ data/val/ --recursive --quiet

In [ ]:
%%bash
# Run training locally on GPU using docker run
mkdir -p local_training_output

docker run --rm \
    --gpus all \
    -v $(pwd)/source_dir:/opt/ml/code \
    -v $(pwd)/data/train:/opt/ml/input/data/train \
    -v $(pwd)/data/val:/opt/ml/input/data/val \
    -v $(pwd)/local_training_output:/opt/training \
    -w /opt/ml/code \
    tf2-object-detection \
    /bin/bash run_training.sh \
    --model_dir /opt/training \
    --pipeline_config_path pipeline.config \
    --num_train_steps 20000 \
    --sample_1_of_n_eval_examples 1

Using fine_tune_checkpoint: checkpoint/ckpt-0
===TRAINING THE MODEL==


2026-01-28 15:28:37.392205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-28 15:28:37.392248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-28 15:28:37.393457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-28 15:28:37.399336: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-28 15:28:40.525738: I external/local_xla/xla/

==EVALUATING THE MODEL==


2026-01-28 18:45:25.226465: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-28 18:45:25.226509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-28 18:45:25.227748: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-28 18:45:25.233854: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-28 18:45:27.686598: E external/local_xla/xla/

creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.45s).
Accumulating evaluation results...
DONE (t=0.11s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.351
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.407
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.583
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.028
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.141
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.206
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.149
 Av

2026-01-28 18:47:32.764589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-28 18:47:32.764635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-28 18:47:32.765883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-28 18:47:32.771896: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-28 18:47:35.117686: E external/local_xla/xla/

In [18]:
import os
import glob
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

def plot_experiment_metrics(model_name, base_dir='experiments'):
    """
    Parses TensorBoard event files and plots Training vs Evaluation Loss.
    """
    model_dir = os.path.join(base_dir, model_name)
    train_dir = os.path.join(model_dir, 'train')
    eval_dir = os.path.join(model_dir, 'eval')
    
    # Find event files
    try:
        train_event_file = glob.glob(os.path.join(train_dir, 'events.out.tfevents*'))[0]
        eval_event_file = glob.glob(os.path.join(eval_dir, 'events.out.tfevents*'))[0]
    except IndexError:
        print(f"Could not find event files in {model_dir}. Did you run training and evaluation?")
        return

    # Load data
    print(f"Loading logs for {model_name}...")
    ea_train = EventAccumulator(train_event_file)
    ea_train.Reload()
    ea_eval = EventAccumulator(eval_event_file)
    ea_eval.Reload()
    
    # Extract scalar data
    # Note: 'Loss/total_loss' is the standard tag for TF2 OD API
    try:
        train_steps = [s.step for s in ea_train.Scalars('Loss/total_loss')]
        train_loss = [s.value for s in ea_train.Scalars('Loss/total_loss')]
        eval_steps = [s.step for s in ea_eval.Scalars('Loss/total_loss')]
        eval_loss = [s.value for s in ea_eval.Scalars('Loss/total_loss')]
    except KeyError:
        print("Could not find 'Loss/total_loss' in logs. Keys available:", ea_train.Scalars().keys())
        return

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(train_steps, train_loss, label='Training Loss', alpha=0.6)
    plt.plot(eval_steps, eval_loss, label='Evaluation Loss', linewidth=3, color='red')
    
    plt.title(f'Learning Curve: {model_name}')
    plt.xlabel('Steps')
    plt.ylabel('Total Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Run this for your specific model folder name
# Replace 'efficientdet_d1' with the actual folder name inside your 'experiments' directory
# You can list them with: !ls experiments
plot_experiment_metrics('efficientdet_d1') 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 import os                                                                                   │
│    2 import glob                                                                                 │
│    3 import matplotlib.pyplot as plt                                                             │
│ ❱  4 from tensorboard.backend.event_processing.event_accumulator import EventAccumulator         │
│    5                                                                                             │
│    6 def plot_experiment_metrics(model_name, base_dir='experiments'):                            │
│    7 │   """                                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'tensorboard'

## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [ ]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir": "/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "20000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='local_gpu',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the write-up.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your write-up), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [ ]:
# your write-up goes here.